# Create UC Functions

In [0]:
%pip install -r ../requirements.txt --quiet
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from shared.helper import *

in_workspace = is_running_in_databricks()
global_config = get_global_config(path='../global_config/databricks_config.yaml')

if not in_workspace:
    from databricks.connect import DatabricksSession
    from dotenv import load_dotenv
    import os

    print("Running the notebook locally ...")
    spark = DatabricksSession.builder.getOrCreate()
    load_dotenv('../.env')

    DATABRICKS_HOST = os.getenv('host')
    DATABRICKS_TOKEN = os.getenv('token')
else:
    print("Running the notebook in Databricks workspace ...")
    DATABRICKS_HOST = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

print(f"host: {DATABRICKS_HOST}")

In [0]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = global_config['catalog']
SCHEMA = global_config['schema_insurance_agents']
print(f"Catalog: {CATALOG} | Schema: {SCHEMA}")

# Create UC Tools

## `search_claims_details_by_policy_no()`

In [0]:
sql_function = \
f"""
create or replace function {CATALOG}.{SCHEMA}.search_claims_details_by_policy_no(
  input_policy_no STRING comment 'policy Number'
)
RETURNS table
COMMENT 'Returns claims details about a customer given policy number'
RETURN
SELECT *
FROM {CATALOG}.{SCHEMA}.claims_table
WHERE policy_no = input_policy_no
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
(spark.sql(f"select * from {CATALOG}.{SCHEMA}.search_claims_details_by_policy_no('102070455')")
 .show
 (truncate=False))

## `policy_doc_vector_search()`

In [0]:
sql_function = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.policy_doc_vector_search(
  query STRING comment 'The query string for searching insurance policy documentation.'
)
RETURNS TABLE
COMMENT 'Executes a search on insurance policy documentation to retrieve text documents most relevant to the input query.'
RETURN
SELECT
  chunked_text as page_content,
  map('doc_path', path, 'chunk_id', chunk_id) as metadata
FROM
  vector_search(
    index => '{CATALOG}.agents.policy_docs_chunked_files_vs_index',
    query => query,
    num_results => 3
  )
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
(spark.sql(f"select * from {CATALOG}.{SCHEMA}.policy_doc_vector_search('car accident coverage')")
 .show())